<a href="https://colab.research.google.com/github/melihilerialkan/BachelorFingerprintKlassifikation/blob/main/classifikation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from tensorflow.keras import Sequential, models 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [31]:
#Fuer welcher Finger an welcher Hand (10 Klasse)
def label_extrahieren(img_path):
    name, _ = os.path.splitext(os.path.basename(img_path))

    id, usw = name.split('__')

    Geschlecht, richtung, finger, _,_ = usw.split('_')

    if richtung == 'Left':

      if finger == 'thumb':
          finger = 0
      elif finger == 'index':
          finger = 1
      elif finger == 'middle':
          finger = 2
      elif finger == 'ring':
          finger = 3
      elif finger == 'little':
          finger = 4
    else:
      if finger == 'thumb':
          finger = 5
      elif finger == 'index':
          finger = 6
      elif finger == 'middle':
          finger = 7
      elif finger == 'ring':
          finger = 8
      elif finger == 'little':
          finger = 9


    return np.array([finger], dtype=np.uint16)

In [2]:
#Fuer welcher Finger an welcher Hand (10 Klasse)
def test_label_extrahieren(img_path):
    name, _ = os.path.splitext(os.path.basename(img_path))

    id, usw = name.split('__')

    Geschlecht, richtung, finger, _ = usw.split('_')

    if richtung == 'Left':

      if finger == 'thumb':
          finger = 0
      elif finger == 'index':
          finger = 1
      elif finger == 'middle':
          finger = 2
      elif finger == 'ring':
          finger = 3
      elif finger == 'little':
          finger = 4
    else:
      if finger == 'thumb':
          finger = 5
      elif finger == 'index':
          finger = 6
      elif finger == 'middle':
          finger = 7
      elif finger == 'ring':
          finger = 8
      elif finger == 'little':
          finger = 9


    return np.array([finger], dtype=np.uint16)

In [32]:
img_size = 96
#Function to iterate through all the images
def datenladen(path):
    data = []
    for img in os.listdir(path):
    
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_resize = cv2.resize(img_array, (img_size, img_size))
        label = label_extrahieren(os.path.join(path, img))
        data.append([label[0], img_resize ])
       
    
    return data

In [3]:
img_size = 200
#Function to iterate through all the images
def test_datenladen(path):
    data = []
    for img in os.listdir(path):
    
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_resize = cv2.resize(img_array, (img_size, img_size))
        label = test_label_extrahieren(os.path.join(path, img))
        data.append([label[0], img_resize ])
       
    
    return data

In [33]:
Real_path = "/content/drive/MyDrive/SOCOFing/Altered/Altered-Medium"
Easy_path = "/content/drive/MyDrive/test"


In [34]:
data = datenladen(Real_path)
random.shuffle(data)
X, y = [], []
for label, feature in data:
    y.append(label)
    X.append(feature)
X = np.array(X).reshape(-1, img_size, img_size, 1)
X = X / 255.0
y = to_categorical(y, num_classes = 10)

In [17]:
test_path = "/content/drive/MyDrive/input"
# Easy_path = "/test"
test = test_datenladen(test_path)
X_test, y_test = [], []

for label, feature in test:
    X_test.append(feature)
    y_test.append(label)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 1)
X_test = X_test / 255.0
y_test = to_categorical(y_test, num_classes = 10)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 13)

In [40]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential([
                    Conv2D(32, 3, padding='same', activation='relu',kernel_initializer='he_uniform', input_shape = [96, 96, 1]),
                    MaxPooling2D(2),
                    Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation='relu'),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(10, activation = 'softmax'),
                    ])

In [41]:
model.compile(optimizer = optimizers.Adam(1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [42]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 50, 
          validation_data = (X_val,y_val), callbacks = [early_stopping_cb], verbose = 1)

Epoch 1/50
94/94 [==============================] - 108s 1s/step - loss: 2.2847 - accuracy: 0.2161 - val_loss: 1.6744 - val_accuracy: 0.4066
Epoch 2/50
94/94 [==============================] - 107s 1s/step - loss: 1.5783 - accuracy: 0.4516 - val_loss: 1.4849 - val_accuracy: 0.4817
Epoch 3/50
94/94 [==============================] - 107s 1s/step - loss: 1.3968 - accuracy: 0.5130 - val_loss: 1.3701 - val_accuracy: 0.5157
Epoch 4/50
94/94 [==============================] - 107s 1s/step - loss: 1.2607 - accuracy: 0.5671 - val_loss: 1.2213 - val_accuracy: 0.5764
Epoch 5/50
94/94 [==============================] - 107s 1s/step - loss: 1.1599 - accuracy: 0.6022 - val_loss: 1.1555 - val_accuracy: 0.5964
Epoch 6/50
94/94 [==============================] - 107s 1s/step - loss: 1.0759 - accuracy: 0.6430 - val_loss: 1.1248 - val_accuracy: 0.6118
Epoch 7/50
94/94 [==============================] - 107s 1s/step - loss: 1.0062 - accuracy: 0.6669 - val_loss: 1.1362 - val_accuracy: 0.5975
Epoch 8/50
94

In [10]:
pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [11]:
pred = to_categorical(pred, num_classes = 10)

In [12]:
pred[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [13]:
y_test[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)